In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import col

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

#!hdfs dfs -ls
!hdfs dfs -copyFromLocal Asistencia.csv /tmp/
!hdfs dfs -copyFromLocal Evaluacion.csv /tmp/

In [3]:
!hdfs dfs -ls /tmp/*csv

-rw-r--r--   1 root supergroup      93121 2021-05-28 17:56 /tmp/Asistencia.csv
-rw-r--r--   1 root supergroup     111944 2021-05-28 17:56 /tmp/Evaluacion.csv


In [4]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...
asis = SesionSpark.read.load('/tmp/Asistencia.csv',format="csv",sep=',',inferSchema='true',header='true')
eva = SesionSpark.read.load('/tmp/Evaluacion.csv',format="csv",sep=',',inferSchema='true',header='true')

In [5]:
#Validamos datos cambiando el nombre de las columnas
asis1 = asis.withColumnRenamed("Marca temporal", "tiempo")
asis2 = asis1.withColumnRenamed("Nombre de usuario", "usuario")
asis3 = asis2.withColumnRenamed("Equipo al que perteneces:", "equipoPertenece")
asis4 = asis3.withColumnRenamed("Equipo que va a exponer:", "equipoExponer")
asis4.printSchema()
asis = asis4

root
 |-- tiempo: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- equipoPertenece: string (nullable = true)
 |-- equipoExponer: string (nullable = true)



In [6]:
eva2 = eva.withColumnRenamed("Marca temporal","tiempo") \
    .withColumnRenamed("Nombre de usuario","usuario") \
    .withColumnRenamed("Equipo que vas a evaluar:","equipoEvaluar") \
    .withColumnRenamed("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?","ventajas") \
    .withColumnRenamed("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?","problema") \
    .withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","introduccion") \
    .withColumnRenamed("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?","solucion") \
    .withColumnRenamed("Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.","traccion") \
    .withColumnRenamed("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ","modeloNegocio") \
    .withColumnRenamed("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?","inversion") \
    .withColumnRenamed("Exposición: ¿Qué tan coordinados estaban los expositores?","exposicion1") \
    .withColumnRenamed("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?","exposicion2") \
    .withColumnRenamed("Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.","exposicion3") \
    .withColumnRenamed("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)","supon") \
    .withColumnRenamed("Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.","observacion") \
    .withColumnRenamed("Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.","contacto") \
    .withColumnRenamed("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?","competencia") \
    .withColumnRenamed("Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.","mercado") \
    .withColumnRenamed("Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.","producto") \
    .withColumnRenamed("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?", "equipo")
eva = eva2
eva2.printSchema()

root
 |-- tiempo: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- equipoEvaluar: string (nullable = true)
 |-- introduccion: integer (nullable = true)
 |-- equipo: integer (nullable = true)
 |-- problema: integer (nullable = true)
 |-- ventajas: integer (nullable = true)
 |-- solucion: integer (nullable = true)
 |-- producto: integer (nullable = true)
 |-- traccion: integer (nullable = true)
 |-- mercado: integer (nullable = true)
 |-- competencia: integer (nullable = true)
 |-- modeloNegocio: integer (nullable = true)
 |-- inversion: integer (nullable = true)
 |-- contacto: integer (nullable = true)
 |-- exposicion1: integer (nullable = true)
 |-- exposicion2: integer (nullable = true)
 |-- exposicion3: integer (nullable = true)
 |-- supon: string (nullable = true)
 |-- observacion: string (nullable = true)



In [7]:
#Datos adicionales
totalAsis = asis.count()
totalEva = eva.count()
print('Total de asistencias: ', totalAsis, ' y total de evaluaciones: ', totalEva, '.')

Total de asistencias:  662  y total de evaluaciones:  617 .


In [184]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

numUsuarios = asis.sort(['usuario']).select(['usuario']).drop_duplicates().count()
#Respuesta
asis.sort(['usuario']).select(['usuario']).drop_duplicates().show(numUsuarios, truncate = False)

+-------------------------+
|usuario                  |
+-------------------------+
|abanolc@unal.edu.co      |
|acastrillonv@unal.edu.co |
|aflemag@unal.edu.co      |
|anapariciom@unal.edu.co  |
|angutierrezb@unal.edu.co |
|anoriega@unal.edu.co     |
|auarbelaeza@unal.edu.co  |
|bocampo@unal.edu.co      |
|cgiraldo@unal.edu.co     |
|cjfunezg@unal.edu.co     |
|cquinchiar@unal.edu.co   |
|dadazam@unal.edu.co      |
|daestradam@unal.edu.co   |
|davgarciava@unal.edu.co  |
|dballesteroso@unal.edu.co|
|dbrito@unal.edu.co       |
|dcadavid@unal.edu.co     |
|dcardonaal@unal.edu.co   |
|dchavarriar@unal.edu.co  |
|dgarciabl@unal.edu.co    |
|dgerenal@unal.edu.co     |
|dgiraldolo@unal.edu.co   |
|dguardia@unal.edu.co     |
|dosoriom@unal.edu.co     |
|eapenad@unal.edu.co      |
|elondonoc@unal.edu.co    |
|emflorezb@unal.edu.co    |
|emlopezr@unal.edu.co     |
|eporrasm@unal.edu.co     |
|esgarciac@unal.edu.co    |
|favasquezj@unal.edu.co   |
|fguerrerot@unal.edu.co   |
|fmiranda@unal.edu.c

In [185]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

#Creamos una vista para trabajar con SQL
asis.createOrReplaceTempView('vistaAsis')

from pyspark.sql import functions as F

#Respuesta
SesionSpark.sql("""
    SELECT usuario, COUNT(usuario) as numAsistencia, MAX(tiempo) as ultima_asistencia, MIN(tiempo) as primera_asistencia
    FROM vistaAsis GROUP BY usuario
""").show(numUsuarios, truncate = False)

+-------------------------+-------------+------------------------------+-------------------------------+
|usuario                  |numAsistencia|ultima_asistencia             |primera_asistencia             |
+-------------------------+-------------+------------------------------+-------------------------------+
|samolinap@unal.edu.co    |10           |2021/04/13 9:37:26 a. m. GMT-5|2021/04/08 8:11:43 a. m. GMT-5 |
|jruedat@unal.edu.co      |8            |2021/04/13 9:41:27 a. m. GMT-5|2021/04/08 8:12:55 a. m. GMT-5 |
|nvalenciat@unal.edu.co   |8            |2021/04/13 9:39:34 a. m. GMT-5|2021/04/08 8:10:18 a. m. GMT-5 |
|serendona@unal.edu.co    |8            |2021/04/13 9:40:25 a. m. GMT-5|2021/04/08 8:10:47 a. m. GMT-5 |
|jgutierrezlo@unal.edu.co |8            |2021/04/13 9:39:06 a. m. GMT-5|2021/04/08 8:12:09 a. m. GMT-5 |
|dgerenal@unal.edu.co     |8            |2021/04/13 9:41:27 a. m. GMT-5|2021/04/08 8:10:03 a. m. GMT-5 |
|cjfunezg@unal.edu.co     |8            |2021/04/13 9:4

In [183]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 SELECT usuario, total FROM (SELECT usuario, COUNT(usuario) AS total FROM vistaAsis WHERE SUBSTRING(equipoPertenece, 15) <> SUBSTRING(equipoExponer, 8) GROUP BY usuario) WHERE total <= 2
""").show(truncate = False)

+---------------------+-----+
|usuario              |total|
+---------------------+-----+
|jdroldano@unal.edu.co|1    |
+---------------------+-----+



In [182]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

#Creamos otra vista para trabajar con SQL
eva.createOrReplaceTempView('vistaEva')

#Respuesta
SesionSpark.sql("""
 SELECT usuario, total FROM (SELECT usuario, COUNT(usuario) AS total FROM vistaAsis WHERE SUBSTRING(equipoPertenece, 15) <> SUBSTRING(equipoExponer, 8) GROUP BY usuario) WHERE total = 0
""").show(truncate = False)

+-------+-----+
|usuario|total|
+-------+-----+
+-------+-----+



In [164]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 SELECT SUBSTRING(equipoPertenece, 15) AS equipos, collect_set(usuario) AS integrantes FROM vistaAsis GROUP BY equipoPertenece
""").show(truncate = False)

+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|equipos                                              |integrantes                                                                                                                                                                                                                                                                                                |
+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [165]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
 SELECT SUBSTRING(equipoPertenece, 15) AS Equipo, COUNT(usuario) AS CantidadAsistentes FROM vistaAsis WHERE equipoPertenece <> equipoExponer GROUP BY equipoPertenece
""").show(truncate = False)

+-----------------------------------------------------+------------------+
|Equipo                                               |CantidadAsistentes|
+-----------------------------------------------------+------------------+
|Gestión de Menús Ejecutivos                          |84                |
|Lo tengo y te lo presto.                             |87                |
|Administración de Inmuebles                          |95                |
|Servicios a domicilio                                |65                |
|Te lo cambio.                                        |89                |
|Gestión de Residencias y Habitaciones Universitarias.|86                |
|Gestión de programas curriculares                    |73                |
|Mi Sanandresito                                      |83                |
+-----------------------------------------------------+------------------+



In [51]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

#Respuesta
SesionSpark.sql("""
   SELECT Asis.usuario, SUBSTRING(Asis.equipoPertenece, 15) AS pertenece, Eva.equipoEvaluar AS evaluo
   FROM vistaAsis Asis INNER JOIN vistaEva Eva ON Asis.usuario = Eva.usuario 
   WHERE Eva.equipoEvaluar = SUBSTRING(Asis.equipoPertenece, 15)
""").show(truncate = False)

+-----------------------+---------------------------------+---------------------------------+
|usuario                |pertenece                        |evaluo                           |
+-----------------------+---------------------------------+---------------------------------+
|daestradam@unal.edu.co |Lo tengo y te lo presto.         |Lo tengo y te lo presto.         |
|anapariciom@unal.edu.co|Servicios a domicilio            |Servicios a domicilio            |
|hriveraa@unal.edu.co   |Gestión de Menús Ejecutivos      |Gestión de Menús Ejecutivos      |
|jruedat@unal.edu.co    |Servicios a domicilio            |Servicios a domicilio            |
|lmayar@unal.edu.co     |Gestión de programas curriculares|Gestión de programas curriculares|
|daestradam@unal.edu.co |Lo tengo y te lo presto.         |Lo tengo y te lo presto.         |
|jruedat@unal.edu.co    |Servicios a domicilio            |Servicios a domicilio            |
|hriveraa@unal.edu.co   |Gestión de Menús Ejecutivos      |G

In [151]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

#Respuesta
notas = """
    SELECT *, ROUND((promIntroduccion + promEquipo + promProblema + promVentajas + promSolucion + 
    promProducto + promTraccion + promMercado + promCompetencia + PromModeloNegiocio + promInversion + 
    promContacto + promExposicion1 + promExposicion2 + promExposicion3)/15,2) AS TOTAL 
    FROM (
        SELECT equipoEvaluar AS equipo,
        ROUND(avg(introduccion),2) AS promIntroduccion,
        ROUND(avg(equipo),2) AS promEquipo,
        ROUND(avg(problema),2) AS promProblema,
        ROUND(avg(ventajas),2) AS promVentajas,
        ROUND(avg(solucion),2) AS promSolucion,
        ROUND(avg(producto),2) AS promProducto,
        ROUND(avg(traccion),2) AS promTraccion,
        ROUND(avg(mercado),2) AS promMercado,
        ROUND(avg(competencia),2) AS promCompetencia,
        ROUND(avg(modeloNegocio),2) AS PromModeloNegiocio,
        ROUND(avg(inversion),2) AS promInversion,
        ROUND(avg(contacto),2) AS promContacto,
        ROUND(avg(exposicion1),2) AS promExposicion1,
        ROUND(avg(exposicion2),2) AS promExposicion2,
        ROUND(avg(exposicion3),2) AS promExposicion3
        FROM (
            SELECT * FROM vistaAsis Asis INNER JOIN vistaEva Eva ON Asis.usuario = Eva.usuario 
            WHERE Eva.equipoEvaluar != SUBSTRING(Asis.equipoPertenece, 15)
        ) GROUP BY equipoEvaluar
    )
""";

SesionSpark.sql(notas).show()

+--------------------+----------------+----------+------------+------------+------------+------------+------------+-----------+---------------+------------------+-------------+------------+---------------+---------------+---------------+-----+
|              equipo|promIntroduccion|promEquipo|promProblema|promVentajas|promSolucion|promProducto|promTraccion|promMercado|promCompetencia|PromModeloNegiocio|promInversion|promContacto|promExposicion1|promExposicion2|promExposicion3|TOTAL|
+--------------------+----------------+----------+------------+------------+------------+------------+------------+-----------+---------------+------------------+-------------+------------+---------------+---------------+---------------+-----+
|Servicios a domic...|            4.49|      4.63|        4.51|        4.27|         4.5|        4.61|         4.1|       4.12|           4.36|              4.27|         4.16|        4.65|           4.49|           4.56|           4.59| 4.42|
|Lo tengo y te lo ...|  

In [152]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

maximos = """SELECT MAX(promIntroduccion) AS maxIntroduccion,
                MAX(promEquipo) AS maxEquipo,
                MAX(promProblema) AS maxProblema,
                MAX(promVentajas) AS maxVentajas,
                MAX(promSolucion) AS maxSolucion,
                MAX(promProducto) AS maxProducto,
                MAX(promTraccion) AS maxTraccion,
                MAX(promMercado) AS maxMercado,
                MAX(promCompetencia) AS maxCompetencia,
                MAX(PromModeloNegiocio) AS maxModeloNegiocio,
                MAX(promInversion) AS maxInversion,
                MAX(promContacto) AS maxContacto,
                MAX(promExposicion1) AS maxExposicion1,
                MAX(promExposicion2) AS maxExposicion2,
                MAX(promExposicion3) AS maxExposicion3,
                MAX(TOTAL) AS maxTOTAL FROM (""" + notas + ")";
            
SesionSpark.sql("SELECT 'Introduccion' AS item, equipo, promIntroduccion AS nota FROM (" + notas + ") WHERE promIntroduccion = (SELECT maxIntroduccion FROM (" + maximos + ")) UNION " +
                    "SELECT 'Equipo' AS item, equipo, promEquipo AS nota FROM (" + notas + ") WHERE promEquipo = (SELECT maxEquipo FROM (" + maximos + ")) UNION " +
                    "SELECT 'Problema' AS item, equipo, promProblema AS nota FROM (" + notas + ") WHERE promProblema = (SELECT maxProblema FROM (" + maximos + ")) UNION " +
                    "SELECT 'Ventajas' AS item, equipo, promVentajas AS nota FROM (" + notas + ") WHERE promVentajas = (SELECT maxVentajas FROM (" + maximos + ")) UNION " +
                    "SELECT 'Solucion' AS item, equipo, promSolucion AS nota FROM (" + notas + ") WHERE promSolucion = (SELECT maxSolucion FROM (" + maximos + ")) UNION " +
                    "SELECT 'Producto' AS item, equipo, promProducto AS nota FROM (" + notas + ") WHERE promProducto = (SELECT maxProducto FROM (" + maximos + ")) UNION " +
                    "SELECT 'Traccion' AS item, equipo, promTraccion AS nota FROM (" + notas + ") WHERE promTraccion = (SELECT maxTraccion FROM (" + maximos + ")) UNION " +
                    "SELECT 'Mercado' AS item, equipo, promMercado AS nota FROM (" + notas + ") WHERE promMercado = (SELECT maxMercado FROM (" + maximos + ")) UNION " +
                    "SELECT 'Competencia' AS item, equipo, promCompetencia AS nota FROM (" + notas + ") WHERE promCompetencia = (SELECT maxCompetencia FROM (" + maximos + ")) UNION " +
                    "SELECT 'ModeloNegiocio' AS item, equipo, PromModeloNegiocio AS nota FROM (" + notas + ") WHERE PromModeloNegiocio = (SELECT maxModeloNegiocio FROM (" + maximos + ")) UNION " +
                    "SELECT 'Inversion' AS item, equipo, promInversion AS nota FROM (" + notas + ") WHERE promInversion = (SELECT maxInversion FROM (" + maximos + ")) UNION " +
                    "SELECT 'Contacto' AS item, equipo, promContacto AS nota FROM (" + notas + ") WHERE promContacto = (SELECT maxContacto FROM (" + maximos + ")) UNION " +
                    "SELECT 'Exposicion1' AS item, equipo, promExposicion1 AS nota FROM (" + notas + ") WHERE promExposicion1 = (SELECT maxExposicion1 FROM (" + maximos + ")) UNION " +
                    "SELECT 'Exposicion2' AS item, equipo, promExposicion2 AS nota FROM (" + notas + ") WHERE promExposicion2 = (SELECT maxExposicion2 FROM (" + maximos + ")) UNION " +
                    "SELECT 'Exposicion3' AS item, equipo, promExposicion3 AS nota FROM (" + notas + ") WHERE promExposicion3 = (SELECT maxExposicion3 FROM (" + maximos + ")) UNION " +
                    "SELECT 'TOTAL' AS item, equipo, TOTAL AS nota FROM (" + notas + ") WHERE TOTAL = (SELECT maxTOTAL FROM (" + maximos + "))"
               ).show(truncate = False)

+--------------+-----------------------------------------------------+----+
|item          |equipo                                               |nota|
+--------------+-----------------------------------------------------+----+
|Producto      |Deserción de estudiantes                             |5.0 |
|Competencia   |Deserción de estudiantes                             |5.0 |
|Problema      |Deserción de estudiantes                             |5.0 |
|Exposicion2   |Lo tengo y te lo presto.                             |4.6 |
|Contacto      |Deserción de estudiantes                             |5.0 |
|Inversion     |Gestión de programas curriculares                    |4.54|
|Equipo        |Deserción de estudiantes                             |5.0 |
|Exposicion1   |Te lo cambio.                                        |4.57|
|Solucion      |Deserción de estudiantes                             |4.5 |
|Traccion      |Deserción de estudiantes                             |5.0 |
|Ventajas   

In [153]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...

minimos = """SELECT MIN(promIntroduccion) AS minIntroduccion,
                MIN(promEquipo) AS minEquipo,
                MIN(promProblema) AS minProblema,
                MIN(promVentajas) AS minVentajas,
                MIN(promSolucion) AS minSolucion,
                MIN(promProducto) AS minProducto,
                MIN(promTraccion) AS minTraccion,
                MIN(promMercado) AS minMercado,
                MIN(promCompetencia) AS minCompetencia,
                MIN(PromModeloNegiocio) AS minModeloNegiocio,
                MIN(promInversion) AS minInversion,
                MIN(promContacto) AS minContacto,
                MIN(promExposicion1) AS minExposicion1,
                MIN(promExposicion2) AS minExposicion2,
                MIN(promExposicion3) AS minExposicion3,
                MIN(TOTAL) AS minTOTAL FROM (""" + nota + ")";
            
SesionSpark.sql("SELECT 'Introduccion' AS item, equipo, promIntroduccion AS nota FROM (" + notas + ") WHERE promIntroduccion = (SELECT maxIntroduccion FROM (" + maximos + ")) UNION " +
                    "SELECT 'Equipo' AS item, equipo, promEquipo AS nota FROM (" + notas + ") WHERE promEquipo = (SELECT minEquipo FROM (" + minimos + ")) UNION " +
                    "SELECT 'Problema' AS item, equipo, promProblema AS nota FROM (" + notas + ") WHERE promProblema = (SELECT minProblema FROM (" + minimos + ")) UNION " +
                    "SELECT 'Ventajas' AS item, equipo, promVentajas AS nota FROM (" + notas + ") WHERE promVentajas = (SELECT minVentajas FROM (" + minimos + ")) UNION " +
                    "SELECT 'Solucion' AS item, equipo, promSolucion AS nota FROM (" + notas + ") WHERE promSolucion = (SELECT minSolucion FROM (" + minimos + ")) UNION " +
                    "SELECT 'Producto' AS item, equipo, promProducto AS nota FROM (" + notas + ") WHERE promProducto = (SELECT minProducto FROM (" + minimos + ")) UNION " +
                    "SELECT 'Traccion' AS item, equipo, promTraccion AS nota FROM (" + notas + ") WHERE promTraccion = (SELECT minTraccion FROM (" + minimos + ")) UNION " +
                    "SELECT 'Mercado' AS item, equipo, promMercado AS nota FROM (" + notas + ") WHERE promMercado = (SELECT minMercado FROM (" + minimos + ")) UNION " +
                    "SELECT 'Competencia' AS item, equipo, promCompetencia AS nota FROM (" + notas + ") WHERE promCompetencia = (SELECT minCompetencia FROM (" + minimos + ")) UNION " +
                    "SELECT 'ModeloNegiocio' AS item, equipo, PromModeloNegiocio AS nota FROM (" + notas + ") WHERE PromModeloNegiocio = (SELECT minModeloNegiocio FROM (" + minimos + ")) UNION " +
                    "SELECT 'Inversion' AS item, equipo, promInversion AS nota FROM (" + notas + ") WHERE promInversion = (SELECT minInversion FROM (" + minimos + ")) UNION " +
                    "SELECT 'Contacto' AS item, equipo, promContacto AS nota FROM (" + notas + ") WHERE promContacto = (SELECT minContacto FROM (" + minimos + ")) UNION " +
                    "SELECT 'Exposicion1' AS item, equipo, promExposicion1 AS nota FROM (" + notas + ") WHERE promExposicion1 = (SELECT minExposicion1 FROM (" + minimos + ")) UNION " +
                    "SELECT 'Exposicion2' AS item, equipo, promExposicion2 AS nota FROM (" + notas + ") WHERE promExposicion2 = (SELECT minExposicion2 FROM (" + minimos + ")) UNION " +
                    "SELECT 'Exposicion3' AS item, equipo, promExposicion3 AS nota FROM (" + notas + ") WHERE promExposicion3 = (SELECT minExposicion3 FROM (" + minimos + ")) UNION " +
                    "SELECT 'TOTAL' AS item, equipo, TOTAL AS nota FROM (" + notas + ") WHERE TOTAL = (SELECT minTOTAL FROM (" + minimos + "))"
               ).show(truncate = False)

+--------------+-----------------------------------------------------+----+
|item          |equipo                                               |nota|
+--------------+-----------------------------------------------------+----+
|Equipo        |Mi Sanandresito                                      |3.91|
|Solucion      |Administración de Inmuebles                          |4.19|
|Inversion     |Te lo cambio.                                        |3.59|
|Problema      |Administración de Inmuebles                          |3.97|
|TOTAL         |Administración de Inmuebles                          |4.19|
|ModeloNegiocio|Servicios a domicilio                                |4.27|
|Exposicion1   |Mi Sanandresito                                      |4.28|
|Exposicion2   |Mi Sanandresito                                      |4.18|
|Exposicion3   |Mi Sanandresito                                      |4.13|
|Producto      |Gestión de Menús Ejecutivos                          |4.02|
|Mercado    

In [186]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

SesionSpark.sql("""
   SELECT Asis.usuario, Nt.equipo, TOTAL AS nota
   FROM (SELECT DISTINCT usuario, SUBSTRING(equipoPertenece, 15) AS equipo FROM vistaAsis) Asis 
   INNER JOIN (""" + notas + """) Nt ON Asis.equipo = Nt.equipo
""").show(numUsuarios, truncate = False)

+-------------------------+-----------------------------------------------------+----+
|usuario                  |equipo                                               |nota|
+-------------------------+-----------------------------------------------------+----+
|jruedat@unal.edu.co      |Servicios a domicilio                                |4.42|
|cjfunezg@unal.edu.co     |Servicios a domicilio                                |4.42|
|jdroldano@unal.edu.co    |Servicios a domicilio                                |4.42|
|dosoriom@unal.edu.co     |Servicios a domicilio                                |4.42|
|jppinedal@unal.edu.co    |Servicios a domicilio                                |4.42|
|sacastrot@unal.edu.co    |Servicios a domicilio                                |4.42|
|bocampo@unal.edu.co      |Servicios a domicilio                                |4.42|
|anapariciom@unal.edu.co  |Servicios a domicilio                                |4.42|
|jmunozbo@unal.edu.co     |Servicios a domi